# Creating a Cutout with ECMWF ensemble forecast

In this example we download ECMWF ensemble forecast **open-data** for a cutout we want to create.

This only works if you have in before

* Installed the ECMWF `earthkit` package
* Documentation on how to use `earthkit` to download data [on their website here](https://earthkit-data.readthedocs.io/en/latest/examples/ecmwf_open_data.html)

Open-data only contains a subset of forecast variables.
As a result, there are missing features in this cutout which we replace with the recent 3-year mean from ERA5.

In [1]:
import gc
import pandas as pd
import earthkit.data as ekd
import dask.array as da
import xarray as xr


step_schedule = list(range(0, 366, 6))
model_config = dict(
    name="ecmwf-open-data",
    stream="enfo",
    type="pf",
    model="ifs",
    source="ecmwf",
)


def formate_forecast_to_analysis(
    forecast: xr.Dataset
):
    assert "longitude" in forecast.coords, "Dataset must have a longitude coordinate"
    assert forecast.longitude.min() >= -180 and forecast.longitude.max() <= 180, "Longitude values must be in the range [-180, 180]"
    return (
        # convert longitudes from [-180, 180] to [0, 360]
        forecast.assign_coords(longitude=(((forecast.longitude + 360) % 360)))
    )

def formate_analysis_to_forecast(
    analysis: xr.Dataset
):
    assert "longitude" in analysis.coords, "Dataset must have a longitude coordinate"
    assert analysis.longitude.min() >= 0 and analysis.longitude.max() <= 360, "Longitude values must be in the range [0, 360]"
    return (
        # 1. convert longitudes from [0, 360] to [-180, 180]
        analysis.assign_coords(longitude=(((analysis.longitude + 180) % 360) - 180))
    )
    

In [2]:
forecast_date = "2025-09-26"
forecast_time = 0
steps = [18, 24]

In [3]:
forecast_datetime = pd.to_datetime(forecast_date) + pd.Timedelta(hours=forecast_time)
reference_datetimes = [forecast_datetime + pd.Timedelta(hours=s) for s in steps]
assert all(s in step_schedule for s in steps), f"Step {steps} not in schedule {step_schedule}"

## Height

Geopotential: z

In [4]:
ds = ekd.from_source(
    "ecmwf-open-data",
    date=forecast_date,
    time=forecast_time,
    param="z",
    levtype="sfc",
    step=0,
    model="ifs",
    source="ecmwf",
).to_xarray()

def _add_height(ds):
    g0 = 9.80665
    z = ds["z"]
    return z / g0

height = _add_height(ds)

By downloading data from the ECMWF open data dataset, you agree to the terms: Attribution 4.0 International (CC BY 4.0). Please attribute ECMWF when downloading this data.


In [5]:
height

<xarray.DataArray 'z' (latitude: 721, longitude: 1440)> Size: 8MB
array([[ 1.91893862e-01,  1.91893862e-01,  1.91893862e-01, ...,
         1.91893862e-01,  1.91893862e-01,  1.91893862e-01],
       [ 1.10963845e+00,  1.00766683e+00,  1.00766683e+00, ...,
         1.10963845e+00,  1.10963845e+00,  1.10963845e+00],
       [-5.21907488e-01, -5.21907488e-01, -5.21907488e-01, ...,
        -4.19935866e-01, -4.19935866e-01, -4.19935866e-01],
       ...,
       [ 2.89985692e+03,  2.89924509e+03,  2.89863326e+03, ...,
         2.90159043e+03,  2.90097860e+03,  2.90036678e+03],
       [ 2.80981598e+03,  2.80971400e+03,  2.80951006e+03, ...,
         2.81052978e+03,  2.81032583e+03,  2.81001992e+03],
       [ 2.76443860e+03,  2.76443860e+03,  2.76443860e+03, ...,
         2.76443860e+03,  2.76443860e+03,  2.76443860e+03]],
      shape=(721, 1440))
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8

## Wind

In [6]:
ds = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["10u", "10v", "100u", "100v"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()

In [7]:
wind_spd = {}
for h in [10, 100]:
    w = da.sqrt(ds[f"{h}u"]**2 + ds[f"{h}v"]**2)
    w = w.assign_attrs(
            units=ds[f"{h}u"].attrs["units"], long_name=f"{h} metre wind speed"
        )
    wind_spd[h] = w

In [8]:
wind_spd[100]

<xarray.DataArray (number: 50, step: 2, latitude: 721, longitude: 1440)> Size: 831MB
array([[[[11.98380428, 11.98380428, 11.98380428, ..., 11.98380428,
          11.98380428, 11.98380428],
         [12.9039216 , 12.89664679, 12.8895786 , ..., 12.89553096,
          12.89736917, 12.8995646 ],
         [13.07076009, 13.06677971, 13.06307146, ..., 13.06172196,
          13.06448863, 13.0675014 ],
         ...,
         [ 8.20075096,  8.18680298,  8.17135875, ...,  8.24720405,
           8.23155384,  8.21749475],
         [ 6.47849151,  6.47475443,  6.46979398, ...,  6.48067767,
           6.47935418,  6.47958557],
         [ 7.63498559,  7.63498559,  7.63498559, ...,  7.63498559,
           7.63498559,  7.63498559]],

        [[15.2960405 , 15.2960405 , 15.2960405 , ..., 15.2960405 ,
          15.2960405 , 15.2960405 ],
         [16.38140561, 16.37216455, 16.36125467, ..., 16.36051955,
          16.36782485, 16.37348712],
         [14.51760116, 14.51423903, 14.51111297, ..., 14.4586789 ,
          14.47938548, 14.49824995],
...
         [ 9.75328008,  9.73554738,  9.719325  , ...,  9.78883222,
           9.77711619,  9.76548806],
         [ 7.94574239,  7.9409835 ,  7.93637857, ...,  7.95343245,
           7.95048434,  7.94767338],
         [ 9.69126324,  9.69126324,  9.69126324, ...,  9.69126324,
           9.69126324,  9.69126324]],

        [[10.55962688, 10.55962688, 10.55962688, ..., 10.55962688,
          10.55962688, 10.55962688],
         [11.94510125, 11.93213433, 11.91930723, ..., 11.97914911,
          11.96826723, 11.95562325],
         [13.70118646, 13.68414761, 13.66733118, ..., 13.73553652,
          13.72385562, 13.71240541],
         ...,
         [ 9.562163  ,  9.54490136,  9.52772536, ...,  9.60982191,
           9.59385813,  9.5779717 ],
         [ 7.31233491,  7.31167255,  7.31116459, ...,  7.30926356,
           7.31033744,  7.30991401],
         [10.49219775, 10.49219775, 10.49219775, ..., 10.49219775,
          10.49219775, 10.49219775]]]], shape=(50, 2, 721, 1440))
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Attributes:
    units:      m s**-1
    long_name:  100 metre wind speed

In [9]:
wnd_shear_exp = (
    da.log(wind_spd[10] / wind_spd[100]) / da.log(10 / 100)
).assign_attrs(units="", long_name="wind shear exponent")
wnd_shear_exp

<xarray.DataArray (number: 50, step: 2, latitude: 721, longitude: 1440)> Size: 831MB
array([[[[0.11513476, 0.11513476, 0.11513476, ..., 0.11513476,
          0.11513476, 0.11513476],
         [0.11412909, 0.11385801, 0.11357258, ..., 0.11390825,
          0.11395512, 0.11401362],
         [0.11717157, 0.11688078, 0.11664659, ..., 0.11661102,
          0.11657336, 0.11721233],
         ...,
         [0.17427189, 0.17379923, 0.17397589, ..., 0.17473018,
          0.17490214, 0.17415975],
         [0.18953216, 0.1899159 , 0.19020708, ..., 0.18980482,
          0.18894544, 0.18960549],
         [0.17055186, 0.17055186, 0.17055186, ..., 0.17055186,
          0.17055186, 0.17055186]],

        [[0.21190571, 0.21190571, 0.21190571, ..., 0.21190571,
          0.21190571, 0.21190571],
         [0.19574897, 0.19546665, 0.19575222, ..., 0.19545255,
          0.19563544, 0.19556652],
         [0.13149928, 0.1318076 , 0.13198247, ..., 0.12860598,
          0.12965635, 0.13064321],
...
         [0.2170885 , 0.21629818, 0.21656723, ..., 0.21782251,
          0.2177281 , 0.21721128],
         [0.25357825, 0.25403164, 0.25448479, ..., 0.25319181,
          0.25376871, 0.25434473],
         [0.1969891 , 0.1969891 , 0.1969891 , ..., 0.1969891 ,
          0.1969891 , 0.1969891 ]],

        [[0.10658645, 0.10658645, 0.10658645, ..., 0.10658645,
          0.10658645, 0.10658645],
         [0.09465974, 0.0947921 , 0.09492494, ..., 0.09480395,
          0.09502862, 0.09518495],
         [0.09911284, 0.09913742, 0.0991654 , ..., 0.0997706 ,
          0.09933595, 0.09954699],
         ...,
         [0.17431963, 0.17416822, 0.17438388, ..., 0.17484698,
          0.17412493, 0.17404146],
         [0.20358852, 0.20451017, 0.20543504, ..., 0.20374738,
          0.20337932, 0.20433509],
         [0.18920192, 0.18920192, 0.18920192, ..., 0.18920192,
          0.18920192, 0.18920192]]]], shape=(50, 2, 721, 1440))
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Attributes:
    units:      
    long_name:  wind shear exponent

In [10]:
azimuth = da.arctan2(ds["100u"], ds["100v"])
wnd_azimuth = azimuth.where(azimuth >= 0, azimuth + 2 * da.pi)
wnd_azimuth

<xarray.DataArray '100u' (number: 50, step: 2, latitude: 721, longitude: 1440)> Size: 831MB
array([[[[3.05050843, 3.05050843, 3.05050843, ..., 3.05050843,
          3.05050843, 3.05050843],
         [1.65554884, 1.65954919, 1.66355399, ..., 1.64344979,
          1.64738774, 1.65147542],
         [1.69924912, 1.70381125, 1.70837607, ..., 1.68623185,
          1.69057291, 1.69491206],
         ...,
         [5.37009408, 5.36749379, 5.36502978, ..., 5.37773576,
          5.37530913, 5.37272746],
         [5.53527684, 5.53102981, 5.52699751, ..., 5.54784955,
          5.5435919 , 5.53953669],
         [4.20884531, 4.20884531, 4.20884531, ..., 4.20884531,
          4.20884531, 4.20884531]],

        [[2.92812939, 2.92812939, 2.92812939, ..., 2.92812939,
          2.92812939, 2.92812939],
         [1.51912612, 1.52327741, 1.52742815, ..., 1.50614506,
          1.51047813, 1.51480058],
         [1.47807931, 1.48211143, 1.48614534, ..., 1.46453184,
          1.46902429, 1.47362481],
...
         [5.46572687, 5.46290066, 5.45992753, ..., 5.47480573,
          5.47181093, 5.46880898],
         [5.57654033, 5.57213971, 5.5677339 , ..., 5.58892456,
          5.58477284, 5.58061811],
         [4.20650901, 4.20650901, 4.20650901, ..., 4.20650901,
          4.20650901, 4.20650901]],

        [[3.08005989, 3.08005989, 3.08005989, ..., 3.08005989,
          3.08005989, 3.08005989],
         [1.68440043, 1.68782005, 1.69124708, ..., 1.6727606 ,
          1.67662742, 1.6805191 ],
         [1.68728139, 1.69130802, 1.69534464, ..., 1.67497143,
          1.67906788, 1.68317124],
         ...,
         [5.61623287, 5.61324293, 5.61024219, ..., 5.624765  ,
          5.62193039, 5.61908634],
         [5.68494556, 5.68035088, 5.67575546, ..., 5.69754037,
          5.6934645 , 5.68924039],
         [4.31171396, 4.31171396, 4.31171396, ..., 4.31171396,
          4.31171396, 4.31171396]]]], shape=(50, 2, 721, 1440))
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Attributes:
    param:          100u
    standard_name:  eastward_wind
    long_name:      100 metre U wind component
    paramId:        228246
    units:          m s**-1
    _earthkit:      {'message': b'GRIB\xff\xff\x00\x02\x00\x00\x00\x00\x00\x0...

In [11]:
del ds
gc.collect()

2410

`fsr` (Forecast surface roughness) is not available in `ecmwf-opendata` so we will specify `extrapolate_wind_speed` with `method='power'` to avoid the usage of `fsr`

## Influx

In [19]:
ds = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["ssrd", "ssr"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (number: 50, step: 2, latitude: 721, longitude: 1440)
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    ssr        (number, step, latitude, longitude) float64 831MB ...
    ssrd       (number, step, latitude, longitude) float64 831MB ...
Attributes:
    class:        od
    stream:       enfo
    levtype:      sfc
    type:         pf
    expver:       0001
    date:         20250926
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

Again, since `ecmwf-opendata` does not contain `fdir` and `tisr`, we use the 3-year average from ERA5.

In [14]:
import cdsapi

missing_influx_vars = {
    "tisr": "toa_incident_solar_radiation",
    "fdir": "total_sky_direct_solar_radiation_at_surface",
}

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": list(missing_influx_vars.values()),
    "year": list(set([str(ref_dt.year - i) for ref_dt in reference_datetimes for i in range(1, 4)])),
    "month": list(set([str(ref_dt.month) for ref_dt in reference_datetimes])),
    "day": list(set([str(ref_dt.day) for ref_dt in reference_datetimes])),
    "time": [f"{ref_dt.hour:02d}:00" for ref_dt in reference_datetimes],
    "data_format": "grib",
    "download_format": "unarchived"
}

client = cdsapi.Client()
result = client.retrieve(dataset, request)
result.download("climo_influx.grib")

2025-09-28 13:07:45,420 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-28 13:07:45,420 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-28 13:07:45,701 INFO Request ID is 58be56c9-b89e-4ef0-898f-9e69c976398e
2025-09-28 13:07:45,762 INFO status has been updated to accepted
2025-09-28 13:07:59,285 INFO status has been updated to running
2025-09-28 13:08:06,928 INFO status has been updated to successful


'climo_influx.grib'

In [15]:
import xarray as xr

ds_influx = (
    xr.open_dataset("climo_influx.grib", decode_timedelta=True, engine="cfgrib")
    .rename({"fdir": "influx_direct", "tisr": "influx_toa"})
)
ds_influx = (
    ds_influx
    # convert longitudes from [0, 360] to [-180, 180]
    .assign_coords(longitude=(((ds_influx.longitude + 180) % 360) - 180))
    .sortby("longitude")
)
ds_influx

Ignoring index file 'climo_influx.grib.5b7b6.idx' older than GRIB file


<xarray.Dataset> Size: 199MB
Dimensions:        (time: 12, step: 2, latitude: 721, longitude: 1440)
Coordinates:
    number         int64 8B ...
  * time           (time) datetime64[ns] 96B 2022-09-25T18:00:00 ... 2024-09-...
  * step           (step) timedelta64[ns] 16B 06:00:00 12:00:00
    surface        float64 8B ...
  * latitude       (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
    valid_time     (time, step) datetime64[ns] 192B ...
  * longitude      (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    influx_toa     (time, step, latitude, longitude) float32 100MB ...
    influx_direct  (time, step, latitude, longitude) float32 100MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [16]:
climo_influx = (
    ds_influx
    .assign_coords(
        month=ds_influx["valid_time"].dt.month,
        day=ds_influx["valid_time"].dt.day,
        hour=ds_influx["valid_time"].dt.hour,
    )
    .groupby(["month", "day", "hour"])
    .mean()
    .drop_vars("number")
)
climo_influx

<xarray.Dataset> Size: 66MB
Dimensions:        (month: 1, day: 2, hour: 4, latitude: 721, longitude: 1440)
Coordinates:
  * month          (month) int64 8B 9
  * day            (day) int64 16B 26 27
  * hour           (hour) int64 32B 0 6 12 18
    surface        float64 8B 0.0
  * latitude       (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    influx_toa     (latitude, longitude, month, day, hour) float32 33MB 0.0 ....
    influx_direct  (latitude, longitude, month, day, hour) float32 33MB 0.0 ....
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [17]:
ds_forecast = xr.concat(
    [
        (
            climo_influx.sel(
                month=rd.month,
                day=rd.day,
                hour=rd.hour,
            )
            .assign_coords(step=pd.Timedelta(hours=s))
            .drop_vars(["month", "day", "hour"])
        )
        for rd, s in zip(reference_datetimes, steps)
    ],
    dim="step"
)
ds_forecast

<xarray.Dataset> Size: 17MB
Dimensions:        (step: 2, latitude: 721, longitude: 1440)
Coordinates:
    surface        float64 8B 0.0
  * latitude       (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * step           (step) timedelta64[ns] 16B 18:00:00 1 days
Data variables:
    influx_toa     (step, latitude, longitude) float32 8MB 0.0 0.0 ... 1.409e+05
    influx_direct  (step, latitude, longitude) float32 8MB 0.0 0.0 ... 1.958e+04
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
ds_forecast["albedo"] = (
    ((ds["ssrd"] - ds["ssr"]) / ds["ssrd"].where(ds["ssrd"] != 0))
    .fillna(0.0)
    .assign_attrs(units="(0 - 1)", long_name="Albedo")
)

ds_forecast["influx_diffuse"] = (
    (ds["ssrd"] - ds_forecast["influx_direct"])
    .assign_attrs(
        units="J m**-2", long_name="Surface diffuse solar radiation downwards"
    )
)
ds_forecast = xr.concat(
    [
        ds_forecast
        .assign_coords(
            time=forecast_datetime,
        )
    ],
    dim="time"
)


In [21]:
# Convert from energy to power J m**-2 -> W m**-2 and clip negative fluxes
for a in ("influx_direct", "influx_diffuse", "influx_toa"):
    ds_forecast[a] = ds_forecast[a] / (60.0 * 60.0)
    ds_forecast[a].attrs["units"] = "W m**-2"

ds_forecast = (
    ds_forecast
    # convert longitudes from [-180, 180] to [0, 360]
    .assign_coords(longitude=(((ds_forecast.longitude + 360) % 360)))
    # rename the longitude and latitude coordinates to lon and lat
    .rename(
        {
            "latitude": "lat",
            "longitude": "lon",
        }
    )
)
ds_forecast

<xarray.Dataset> Size: 2GB
Dimensions:         (time: 1, step: 2, lat: 721, lon: 1440, number: 50)
Coordinates:
    surface         float64 8B 0.0
  * lat             (lat) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * step            (step) timedelta64[ns] 16B 18:00:00 1 days
  * number          (number) int64 400B 1 2 3 4 5 6 7 8 ... 44 45 46 47 48 49 50
  * time            (time) datetime64[ns] 8B 2025-09-26
  * lon             (lon) float64 12kB 180.0 180.2 180.5 ... 179.2 179.5 179.8
Data variables:
    influx_toa      (time, step, lat, lon) float32 8MB 0.0 0.0 ... 39.13 39.13
    influx_direct   (time, step, lat, lon) float32 8MB 0.0 0.0 0.0 ... 5.44 5.44
    albedo          (time, number, step, lat, lon) float64 831MB 0.0 ... 0.8303
    influx_diffuse  (time, number, step, lat, lon) float64 831MB 0.0 ... 309.7
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

## Temperature

In [ ]:
surface_temp = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["2t", "2d"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=0,
    **model_config
).to_xarray()
surface_temp

In [ ]:
soil_temp = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param="sot", # soil temperature
    levtype="sol",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
soil_temp

## Runoff

In [ ]:
runoff = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param="ro",
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
runoff